In [1]:
import aocd
puzzle = aocd.get_puzzle(year=2024, day=6)

puzzle.examples[0].input_data

'....#.....\n.........#\n..........\n..#.......\n.......#..\n..........\n.#..^.....\n........#.\n#.........\n......#...'

In [2]:
import numpy as np

In [3]:
def parse_input(input_data):
    return np.array([
        list(line) for line in input_data.splitlines()
    ])

parse_input(puzzle.examples[0].input_data)

array([['.', '.', '.', '.', '#', '.', '.', '.', '.', '.'],
       ['.', '.', '.', '.', '.', '.', '.', '.', '.', '#'],
       ['.', '.', '.', '.', '.', '.', '.', '.', '.', '.'],
       ['.', '.', '#', '.', '.', '.', '.', '.', '.', '.'],
       ['.', '.', '.', '.', '.', '.', '.', '#', '.', '.'],
       ['.', '.', '.', '.', '.', '.', '.', '.', '.', '.'],
       ['.', '#', '.', '.', '^', '.', '.', '.', '.', '.'],
       ['.', '.', '.', '.', '.', '.', '.', '.', '#', '.'],
       ['#', '.', '.', '.', '.', '.', '.', '.', '.', '.'],
       ['.', '.', '.', '.', '.', '.', '#', '.', '.', '.']], dtype='<U1')

In [4]:
def guard(grid):
    return np.argwhere((grid == "^") | (grid == "v") | (grid == "<") | (grid == ">"))[0]


movement = {
    "^": np.array((-1, 0)),
    "v": np.array((1, 0)),
    "<": np.array((0, -1)),
    ">": np.array((0, 1)),
}

next_movement = {
    "^": ">",
    "v": "<",
    "<": "^",
    ">": "v",
}


def reachable_positions(start_grid):
    reachable = np.zeros_like(start_grid, dtype=bool)

    current_position = guard(start_grid)
    current_grid = start_grid.copy()

    while True:
        reachable[tuple(current_position)] = True

        next_position = (
            current_position + movement[current_grid[tuple(current_position)]]
        )
        if (
            min(next_position) < 0
            or next_position[0] >= current_grid.shape[0]
            or next_position[1] >= current_grid.shape[1]
        ):
            return reachable

        if current_grid[tuple(next_position)] == "#":
            current_grid[tuple(current_position)] = next_movement[
                current_grid[tuple(current_position)]
            ]
        else:
            current_grid[tuple(next_position)] = current_grid[tuple(current_position)]
            current_grid[tuple(current_position)] = "."
            current_position = next_position

In [5]:
example_input = parse_input(puzzle.examples[0].input_data)

reachable = reachable_positions(example_input)

reachable.sum()

np.int64(41)

In [6]:
for example in puzzle.examples:
    print(
        f"{reachable_positions(parse_input(example.input_data)).sum()=} - {example.answer_a=}"
    )

reachable_positions(parse_input(example.input_data)).sum()=np.int64(41) - example.answer_a='41'


In [7]:
aocd.submit(
    reachable_positions(parse_input(puzzle.input_data)).sum(),
    part="a",
    year=2024,
    day=6,
)

coerced int64 value np.int64(5409) for 2024/06 to '5409'


aocd will not submit that answer again. At 2024-12-09 16:05:38.570534-05:00 you've previously submitted 5409 and the server responded with:
That's the right answer!  You are one gold star closer to finding the Chief Historian. [Continue to Part Two]


In [8]:
import itertools as it
import joblib

direction_index = {
    "^": 0,
    "v": 1,
    "<": 2,
    ">": 3,
}


def results_in_loop(start_grid):
    reachable = np.repeat(np.zeros_like(start_grid, dtype=bool)[None], 4, axis=0)
    current_position = guard(start_grid)
    current_grid = start_grid.copy()

    while True:
        direction = current_grid[tuple(current_position)]

        if reachable[(direction_index[direction], *current_position)]:
            return True
        reachable[(direction_index[direction], *current_position)] = True

        next_position = current_position + movement[direction]
        if (
            min(next_position) < 0
            or next_position[0] >= current_grid.shape[0]
            or next_position[1] >= current_grid.shape[1]
        ):
            return False

        if current_grid[tuple(next_position)] == "#":
            current_grid[tuple(current_position)] = next_movement[direction]
        else:
            current_grid[tuple(next_position)] = current_grid[tuple(current_position)]
            current_grid[tuple(current_position)] = "."
            current_position = next_position


def number_of_grids_with_loop(base_grid):
    def add_obstacle(x, y):
        new_grid = base_grid.copy()
        new_grid[x, y] = "#"
        return new_grid

    x_guard, y_guard = guard(base_grid)
    # new_obstacles = [
    #     (x, y)
    #     for x, y in it.product(range(base_grid.shape[0]), range(base_grid.shape[1]))
    #     if (x, y) != (x_guard, y_guard)
    # ]

    new_obstacles = np.argwhere(reachable_positions(base_grid))

    loops = joblib.Parallel(n_jobs=-1, verbose=1, return_as="generator")(
        joblib.delayed(results_in_loop)(add_obstacle(x, y))
        for x, y in new_obstacles
        if (x, y) != (x_guard, y_guard)
    )
    return sum(loops)

In [9]:
results_in_loop(example_input)

False

In [10]:
for example in puzzle.examples:
    print(
        f"{number_of_grids_with_loop(parse_input(example.input_data))=} - {example.answer_b=}"
    )

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 20 concurrent workers.


number_of_grids_with_loop(parse_input(example.input_data))=6 - example.answer_b='6'


[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:    0.3s finished


In [11]:
num_loops = number_of_grids_with_loop(parse_input(puzzle.input_data))

num_loops

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 20 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 600 tasks      | elapsed:    1.5s
[Parallel(n_jobs=-1)]: Done 2600 tasks      | elapsed:    5.5s
[Parallel(n_jobs=-1)]: Done 5408 out of 5408 | elapsed:   11.4s finished


2022

In [12]:
aocd.submit(num_loops, part='b', year=2024, day=6)

aocd will not submit that answer again. At 2024-12-09 16:32:43.265907-05:00 you've previously submitted 2022 and the server responded with:
That's the right answer!  You are one gold star closer to finding the Chief Historian.You have completed Day 6! You can [Shareon
  Bluesky
Twitter
Mastodon] this victory or [Return to Your Advent Calendar].
